## Extended Metrics
The definition used in this research for extended metrics is to cover anything that is calculated from the basic metrics already covered.  Each extended metric will be covered and will include definition, usage, and a code example.

## Staking Reward
The staking reward is to total OHM to be minted for this epoch.  Each epoch the staking reward calculated below is minted and this would be considered OHM inflation.  Many other extended metrics are based off of staking reward.

In [1]:
from web3 import Web3
import math
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/40f886f8daa347ec901aa7e83d3300de')) # Infura

token_abi = open(f"abis/sohmstaking.abi", "r").read()
contract = w3.eth.contract(Web3.toChecksumAddress('0xFd31c7d00Ca47653c6Ce64Af53c1571f9C36566a'), abi=token_abi)

epoch = contract.functions.epoch().call()
staking_reward = epoch[3] / math.pow(10, 9)
staking_reward

31053.0

## Staking Rebase
The staking rebase is the amount of newly minted OHM distributed per OHM staked.

In [7]:
from web3 import Web3
import math
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/40f886f8daa347ec901aa7e83d3300de')) # Infura

staking_abi = open(f"abis/sohmstaking.abi", "r").read()
staking_contract = w3.eth.contract(Web3.toChecksumAddress('0xFd31c7d00Ca47653c6Ce64Af53c1571f9C36566a'), abi=staking_abi)
token_abi = open(f"abis/sohm.abi", "r").read()
token_contract = w3.eth.contract(Web3.toChecksumAddress('0x04F2694C8fcee23e8Fd0dfEA1d4f5Bb8c352111F'), abi=token_abi)

circ = token_contract.functions.circulatingSupply().call() / math.pow(10, 9)
epoch = staking_contract.functions.epoch().call()

staking_reward = epoch[3] / math.pow(10, 9)
staking_rebase = staking_reward / circ
staking_rebase

0.00575501060350834

## Staking APY
This metric is somewhat obvious hopefully.  To get a value that matches with the Olympus DAO dashboard the staking_apy below needs to be multiplied by 100.

In [9]:
from web3 import Web3
import math
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/40f886f8daa347ec901aa7e83d3300de')) # Infura

staking_abi = open(f"abis/sohmstaking.abi", "r").read()
staking_contract = w3.eth.contract(Web3.toChecksumAddress('0xFd31c7d00Ca47653c6Ce64Af53c1571f9C36566a'), abi=staking_abi)
token_abi = open(f"abis/sohm.abi", "r").read()
token_contract = w3.eth.contract(Web3.toChecksumAddress('0x04F2694C8fcee23e8Fd0dfEA1d4f5Bb8c352111F'), abi=token_abi)

circ = token_contract.functions.circulatingSupply().call() / math.pow(10, 9)
epoch = staking_contract.functions.epoch().call()

staking_reward = epoch[3] / math.pow(10, 9)
staking_rebase = staking_reward / circ
staking_apy = math.pow(1 + staking_rebase, 365 * 3) - 1
staking_apy

53475.26619064352

## 5 Day ROI
The 5 day rate is a good metric to use when trying to determine whether to buy bonds which mature after 5 days or to simply buy and stake.  To get the % shown in the dashboard simply multiple by 100 similar to staking_apy above.

In [10]:
from web3 import Web3
import math
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/40f886f8daa347ec901aa7e83d3300de')) # Infura

staking_abi = open(f"abis/sohmstaking.abi", "r").read()
staking_contract = w3.eth.contract(Web3.toChecksumAddress('0xFd31c7d00Ca47653c6Ce64Af53c1571f9C36566a'), abi=staking_abi)
token_abi = open(f"abis/sohm.abi", "r").read()
token_contract = w3.eth.contract(Web3.toChecksumAddress('0x04F2694C8fcee23e8Fd0dfEA1d4f5Bb8c352111F'), abi=token_abi)

circ = token_contract.functions.circulatingSupply().call() / math.pow(10, 9)
epoch = staking_contract.functions.epoch().call()

staking_reward = epoch[3] / math.pow(10, 9)
staking_rebase = staking_reward / circ
five_day_rate = math.pow(1 + staking_rebase, 5 * 3) - 1
five_day_rate

0.08989692347131362

## Next Rebase

## Staking TVL

## Market Cap

## Percent Staked